In [ ]:
!pip install imageio
!pip install git+https://github.com/tensorflow/docs

In [4]:
import tensorflow as tf
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from  tensorflow.keras import layers
import time
from sklearn.utils import shuffle
from IPython import display

from imutils import build_montages
import numpy as np
import argparse
import cv2

In [5]:
(x_train, y_train) , (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1],x_train.shape[2],1).astype('float32')

# Normalization -1 to 1
normalized_x_train = ( x_train - (np.max(x_train) / 2) ) / (np.max(x_train) / 2)
normalized_x_train = - normalized_x_train

11490434/11490434 [==============================] - 0s 0us/step


In [6]:
((trainX, _), (testX, _)) = tf.keras.datasets.fashion_mnist.load_data()
#((trainX, _), (testX, _)) = tf.keras.datasets.mnist.load_data()
trainImages = np.concatenate([trainX, testX])
trainImages = np.expand_dims(trainImages, axis=-1)
trainImages  = (trainImages.astype("float") - 127.5) / 127.5
buffer_size = trainImages.shape[0]
batch_size = 256
noise_dim = 100
batch_count = np.floor( trainImages.shape[0] / batch_size).astype('int32')

4422102/4422102 [==============================] - 0s 0us/step


In [ ]:
plt.imshow(normalized_x_train[45], cmap='gray')

In [ ]:
# Shuffle data
buffer_size = trainImages.shape[0]
batch_size = 256
batch_count = np.floor( trainImages.shape[0] / batch_size).astype('int32')
shuffled_x_train = tf.data.Dataset.from_tensor_slices(normalized_x_train).shuffle(buffer_size=buffer_size).batch(batch_size=batch_size)

In [14]:
class CGAN:
  
  def __init__(self, buffersize, batchsize):
    self.Generator= None
    self.Discriminator = None
    self.GAN = None
    # This method returns a helper function to compute cross entropy loss
    '''self.cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    self.generator_optimizer = tf.keras.optimizers.Adam(1e-4)
    self.discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)'''

    self.BUFFER_SIZE = buffersize
    self.BATCH_SIZE = batchsize
    self.batchperepochs = np.floor(self.BUFFER_SIZE / self.BATCH_SIZE).astype('int32')
    self.EPOCHS = 50
    self.noise_dim = 100
    self.num_examples_to_generate = 16
    self.lr = 2e-4

    # You will reuse this seed overtime (so it's easier)
    # to visualize progress in the animated GIF)
    self.seed = tf.random.normal([self.num_examples_to_generate, self.noise_dim])

    print("an empty Generative Adversarial Network model is created...!")


  def make_generator(self,dim=7, depth=64, inputDim=100, outputDim=512, channels=1):
    '''
    Make an image Generator  
    '''
    inputShape = (dim, dim, depth)
    chanDim = -1
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(input_dim=inputDim, units=outputDim, use_bias= False, input_shape=(100,)))
    model.add(layers.Activation("relu"))
    model.add(layers.BatchNormalization())

    model.add(layers.Dense(7 * 7 * 64))
    model.add(layers.Activation("relu"))
    model.add(layers.BatchNormalization())
    
    model.add(layers.Reshape(inputShape))
    model.add(layers.Conv2DTranspose(32, (5, 5), strides=(2, 2),padding="same"))
    model.add(layers.Activation("relu"))
    model.add(layers.BatchNormalization(axis=chanDim))

    model.add(layers.Conv2DTranspose(channels, (5, 5), strides=(2, 2),padding="same"))
    model.add(layers.Activation("tanh"))

    print("A Generator is added to the GAN model")
    return model

    
  def make_discriminator(self, alpha=0.2):
    '''
    Make a Discriminator
    '''
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(32, 5 , strides=(2,2), use_bias= False, padding='same', input_shape=[28,28,1]))
    model.add(layers.LeakyReLU())
    #model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    #model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=alpha))

    model.add(layers.Dense(1))
    model.add(layers.Activation("sigmoid"))

    print("A Discriminator is added to the GAN model")
    return model

  def create_GAN(self):
    
    self.Generator = self.make_generator()
    self.Discriminator = self.make_discriminator()
    self.Discriminator.compile(optimizer = tf.keras.optimizers.legacy.Adam(0.001,beta_1=0.5, decay=self.lr / self.EPOCHS),loss="binary_crossentropy")
    self.Discriminator.trainable = False

    ganinput = layers.Input((100,))
    ganoutput = self.Discriminator( self.Generator(ganinput))
    self.GAN = tf.keras.Model(ganinput, ganoutput)
    self.GAN.compile(optimizer= tf.keras.optimizers.legacy.Adam(lr=self.lr, beta_1=0.5, decay=self.lr / self.EPOCHS),loss="binary_crossentropy")
    display.clear_output(wait=True)
    print("A GAN with one generator and one discriminator is created")
    


  def train(self, train_images, epochs):
    print("[INFO] starting training...")
    benchmarkNoise = tf.random.normal((self.BATCH_SIZE, self.noise_dim))
    NUM_EPOCHS = self.EPOCHS
    BATCH_SIZE = self.BATCH_SIZE
    #OUTPUT = args["output"]
    for epoch in range(0,epochs):
      batchperepochs =  self.batchperepochs #train_images.shape[0] / self.BATCH_SIZE
      for j in range(0, batchperepochs):
      #j =0
      #for batch in train_images:
        #j= j+1
        p = None
        noises = tf.random.normal((self.BATCH_SIZE, self.noise_dim))
        generated_images= self.Generator.predict(noises, verbose=0)

        imgs = np.concatenate((train_images[j * self.BATCH_SIZE: (j+1) * self.BATCH_SIZE], generated_images ))
        #imgs = np.concatenate((batch, generated_images ))
        labels = ([1] * self.BATCH_SIZE) + ([0] * self.BATCH_SIZE)
        labels = np.reshape(labels, (-1,))
        (imgs, labels) = shuffle(imgs, labels)

        discloss = self.Discriminator.train_on_batch(imgs,labels)

        noises = tf.random.normal((self.BATCH_SIZE, self.noise_dim))
        fakelabels = ([1] * self.BATCH_SIZE)
        fakelabels = np.reshape(fakelabels, (-1,))

        GANloss = self.GAN.train_on_batch(noises, fakelabels)

        if j== batchperepochs -1:
          p =  ["epoch_{}_2_output.png".format(str(epoch + 1).zfill(4))]
        elif j == np.int32(batchperepochs/2):
          print(j)
          p =  ["epoch_{}_1_output.png".format(str(epoch + 1).zfill(4))]
                
        if p is not None:
          print("[INFO] Step {}_{}: discriminator_loss={:.6f}, "
				    "adversarial_loss={:.6f}".format(epoch + 1, j,
				    	discloss, GANloss))
          images = self.Generator.predict(benchmarkNoise)
          images = ((images * 127.5) + 127.5).astype("uint8")
          images = np.repeat(images, 3, axis=-1)
          vis = build_montages(images, (28, 28), (8, 8))[0]
          p = os.path.sep.join(p)
          cv2.imwrite(p, vis)

    plt.imshow(vis)

  def predict(self, noises):
    predicted_images = self.Generator.predict(noises , verbose=0)
    predicted_images = ((predicted_images * 127.5) + 127.5).astype("uint8")
    predicted_images = np.repeat(predicted_images, 3, axis=-1)
    vis = build_montages(predicted_images, (28, 28), (16, 16))[0]
    return vis
    



In [15]:
cg = CGAN(buffer_size, batch_size)
cg.create_GAN()

A GAN with one generator and one discriminator is created


In [ ]:
cg.train(trainImages,10)

In [ ]:
test_noises = tf.random.normal((batch_size, noise_dim))
image = cg.predict(test_noises)

plt.imshow(image)